In [ ]:
import numpy as np
import argparse
import time
# The mnist library is from https://gist.github.com/akesling/5358964 and provides functions to 
# read MNIST into numpy arrays for processing.
from mnist import read, show

In [ ]:
# read array of tuples of arrays
training_data_orig = list(read(dataset='training', path='.'))

In [ ]:
# reads and binarizes 500 images
training_data = training_data_orig[:500]
labels, imgs = zip(*training_data)
training_binarized = (np.array(imgs)/255 > 0.5).astype(int)
training_binarized = training_binarized*2 - 1

In [ ]:
# introduces 2% noise into each binarized image
training_noisy = np.copy(training_binarized)
# flip 16 values in each image
for i in range(len(training_binarized)):
    for count in range(16):
        row = np.random.randint(0,28)
        col = np.random.randint(0,28)
        training_noisy[i][row][col] = -1 * training_binarized[i][row][col]

In [ ]:
theta_hi_xj = 2
theta_hi_hj = 0.2
def boltzmann(img, c):
    res = np.random.rand(28,28)
    for a in range(200):
        for i in range(len(img)):
            for j in range(len(img[0])):

                inner = 0
                if j > 0:
                    inner += c*(2*res[i][j-1] - 1) + theta_hi_xj*img[i][j-1]

                if j < len(img[0])-1:
                    inner += c*(2*res[i][j+1] - 1) + theta_hi_xj*img[i][j+1]

                if i > 0:
                    inner += c*(2*res[i-1][j] - 1) + theta_hi_xj*img[i-1][j]

                if i < len(img)-1:
                    inner += c*(2*res[i+1][j] - 1) + theta_hi_xj*img[i+1][j]
#                 print(inner)
                res[i][j] = np.exp(inner) / (np.exp(-1*inner) + np.exp(inner))

    return res # table of pi probabilities

In [ ]:
def boltzmann_parallel(imgs,c):
    result = np.empty(np.shape(training_noisy))
    for i in range(len(result)):
        print(i)
        result[i] = boltzmann(imgs[i], c)
    return result

In [ ]:

training_denoised = np.empty(np.shape(training_noisy))
for i in range(len(training_noisy)):
    print(i)
    temp = boltzmann(training_noisy[i], 0.2)
    training_denoised[i] = temp
# map(boltzmann, training_noisy, 0.2)

In [ ]:
# denoise all 500 images in parallel
from multiprocessing import Process, Pool
pool = Pool(7)
results_array = pool.starmap(boltzmann_parallel, [(training_binarized, -1),(training_binarized, -0.6),(training_binarized, -0.2),(training_binarized, 0),(training_binarized, 0.2),(training_binarized, 0.6),(training_binarized, 1)])
# results array = denoised images with values of c= -1, -0.6, -0.2, 0, 0.2(default), 0.6, 1


In [ ]:
# binarize denoised images
training_denoised = results_array[4] # for the default c=0.2 case
training_denoised_bin = np.empty(np.shape(training_noisy))
for i in range(len(training_denoised)):
    for row in range(len(training_denoised[i])):
        for col in range(len(training_denoised[i][0])):
            if training_denoised[i][row][col] > 0.5:
                training_denoised[i][row][col] = 255
                training_denoised_bin[i][row][col] = 1
            else:
                training_denoised[i][row][col] = 0
                training_denoised_bin[i][row][col] = -1

In [ ]:
# calculates different pixels in all images
fraction = np.sum(np.abs(training_denoised_bin - training_binarized)) / (2 * 500 * (28 ** 2))
print(1-fraction) # percentage of correct pixels

In [ ]:
recons = np.abs(training_denoised_bin - training_binarized)
zipped = zip(training_denoised_bin, training_binarized, recons)
zipped_s = sorted(zipped, key=lambda x: np.count_nonzero(x[2]))

#### Most accurate reconstruction

In [ ]:
show(zipped_s[0][1]) # most accurate training image
show(zipped_s[0][0]) # most accurate reconstruction

#### Least accurate reconstruction

In [ ]:
show(zipped_s[-1][1]) # least accurate training image
show(zipped_s[-1][0]) # least accurate reconstruction

## Receiver Operating Curves